## Keras Imports

In [46]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

# set seeds for reproducability
from tensorflow import set_random_seed
from numpy.random import seed
set_random_seed(2)
seed(1)

## Other Imports 

In [47]:
import pickle
import pandas as pd
import numpy as np
import string, os 
from nltk.corpus import stopwords
from nltk.tokenize import TreebankWordTokenizer, sent_tokenize
from collections import Counter
import sys
import json
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import StratifiedShuffleSplit

import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

## Embedding words in a vector space 

In [48]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.array(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


## Reading Lyrics file

In [116]:
for i in range(1,9):
    data_location = 's3://clean-tbd-databucket/clean_training_batch_'+str(i)+'.csv.csv'
    if i==1:
        lyric_df=pd.read_csv(data_location)
    else:
        lyric_df=pd.concat([lyric_df,pd.read_csv(data_location)])
    print(len(lyric_df))
lyric_df = lyric_df[lyric_df['language'] == 'English']
print (len(lyric_df))

25669
36710
56223
64633
103794
120497
136661
152806
141747


## Tokenizing words 

In [117]:
#nltk.download('punkt')
tokenizer = TreebankWordTokenizer()

def tokenize_lyric_summary(summary):
    for sent in sent_tokenize(summary):
        for token in tokenizer.tokenize(sent):
            yield token

UNKNOWN = 'unk'
def build_vocab(data, max_vocab_size=None):
    vocab = Counter()
    total = len(data)
    for i, row in enumerate(data.itertuples()):
        vocab.update(tokenize_lyric_summary(row.lyrics))
        if (i+1)%1000 == 0:
            sys.stdout.write(".")
            sys.stdout.flush()
    final_vocab = {word:i for i, (word, count) in enumerate(vocab.most_common(max_vocab_size))}
    final_vocab[UNKNOWN]=len(final_vocab)+1
    return final_vocab

In [118]:
lyric_tokenizer = build_vocab(lyric_df)
print("Vocab size: ", len(lyric_tokenizer))

.............................................................................................................................................Vocab size:  377699


In [119]:
with open("lyric_tokenizer.json", "w") as f:
    json.dump(lyric_tokenizer, f)
    print("Saved lyric_tokenizer file to lyric_tokenizer.json")

Saved lyric_tokenizer file to lyric_tokenizer.json


In [120]:
with open("lyric_tokenizer.json", "r") as f:
    lyric_tokenizer = json.load(f)
    print("loading lyric_tokenizer file to lyric_tokenizer.json")

loading lyric_tokenizer file to lyric_tokenizer.json


## Creating Embedding matrix 
## Row is represented by word token number
## Columns is the glove embedded vector

In [121]:
embedding_matrix = np.zeros((len(embeddings_index), 100))
cnt = 0
for word, i in lyric_tokenizer.items():
    if cnt > len(embeddings_index) - 1:
        break
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
         embedding_matrix[i] = embedding_vector
    cnt += 1

## Building LSTM model embedding -> LSTM -> Dense -> Softmax -> Output (10 nodes)
## categorial cross entropy

In [122]:
max_sequence_len = 100
embedding_dimension = 100
total_words = np.shape(embedding_matrix)[0]
print (total_words)
NumberGenres = 10

400000


In [123]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len 
    model = Sequential()
    
    # Add Input Embedding Layer
   # model.add(Embedding(total_words, embedding_dimension, 
   #                     weights = [embedding_matrix], input_length=input_len, trainable = False))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100, input_shape=(100,100)))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(NumberGenres, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                1010      
Total params: 81,410
Trainable params: 81,410
Non-trainable params: 0
_________________________________________________________________


## Create Modeling file

In [124]:
lyric_df = pd.concat([lyric_df,pd.get_dummies(lyric_df['genre'], prefix='genre')],axis=1)


In [125]:
lyric_df

,Unnamed: 0,genre,song,clean_lyrics,clean_song,language,lyrics,Hillarylyrics,Satorulyrics,Hillary100words,...,genre_Country,genre_Electronic,genre_Folk,genre_Hip-Hop,genre_Indie,genre_Jazz,genre_Metal,genre_Pop,genre_R&B,genre_Rock
0,0,Country,silver wings,silver wings shine in the sunlight roaring eng...,silver wings,English,Silver wings Shine in the sunlight. Roaring en...,silver wings shine in the sunlight roaring eng...,silver wings shine in the sunlight roaring eng...,"['flight', 'theyre', 'taking', 'you', 'away', ...",...,1,0,0,0,0,0,0,0,0,0
1,1,Country,she even woke me up to say goodbye,mornings come and lord my mind is aching that ...,she even woke me up to say goodbye,English,Morning's come and Lord my mind is aching That...,mornings come and lord my mind is aching that ...,mornings come and lord my mind is aching that ...,"['heart', 'is', 'silently', 'breaking', 'as', ...",...,1,0,0,0,0,0,0,0,0,0
2,2,Country,amazing grace,amazing grace how sweet the sound that saved a...,amazing grace,English,Amazing grace! How sweet the sound That save...,amazing grace how sweet the sound that saved a...,amazing grace how sweet the sound that saved a...,"['like', 'me', 'i', 'once', 'was', 'lost', 'bu...",...,1,0,0,0,0,0,0,0,0,0
3,3,Country,every once in a while,every once in a while i think about you and yo...,every once in a while,English,Every once in a while I think about you And yo...,every once in a while i think about you and yo...,every once in a while i think about you and yo...,"['back', 'of', 'my', 'mind', 'when', 'i', 'hea...",...,1,0,0,0,0,0,0,0,0,0
4,4,Country,5-1-5-0,ever since i met you girl you been on my brain...,NaN,English,"Ever since I met you girl, you been on my brai...",ever since i met you girl you been on my brain...,ever since i met you girl you been on my brain...,"['bout', 'you', 'baby', 'just', 'this', 'side'...",...,1,0,0,0,0,0,0,0,0,0
5,5,Country,hard truth,you dont know who i e am but i know all about ...,hard truth,English,You Don't know who I [E] am But I know all abo...,you dont know who i e am but i know all about ...,you dont know who i e am but i know all about ...,"['could', 'have', 'had', 'it', 'a', 'all', 'a'...",...,1,0,0,0,0,0,0,0,0,0
6,6,Country,after the fire is gone,love is where you find it when you find no lov...,after the fire is gone,English,Love is where you find it When you find no lov...,love is where you find it when you find no lov...,love is where you find it when you find no lov...,"['place', 'again', 'we', 'know', 'its', 'wrong...",...,1,0,0,0,0,0,0,0,0,0
7,7,Country,a house with no curtains,we still were our rings we still say i love yo...,a house with no curtains,English,We still were our rings We still say I love yo...,we still were our rings we still say i love yo...,we still were our rings we still say i love yo...,"['but', 'everyone', 'knows', 'its', 'just', 'a...",...,1,0,0,0,0,0,0,0,0,0
8,8,Country,wanderin',wanderin wanderin mhm my father is an engineer...,wanderin,English,(Wanderin' wanderin' mhm) My father is an eng...,wanderin wanderin mhm my father is an engineer...,wanderin wanderin mhm my father is an engineer...,"['mhm', 'my', 'father', 'is', 'an', 'engineer'...",...,1,0,0,0,0,0,0,0,0,0
9,9,Country,katie bar the door,i could tell the way she held me out on that s...,katie bar the door,English,I could tell the way she held me Out on that s...,i could tell the way she held me out on that s...,i could tell the way she held me out on that s...,"['door', 'is', 'that', 'your', 'husband', 'com...",...,1,0,0,0,0,0,0,0,0,0


In [126]:
def eval_lyric(liststr):
    try:
        return eval(liststr)
    except:
        return []
        

In [127]:
print (len(lyric_df))
lyric_df['Hillary100words'] = lyric_df['Hillary100words'].apply(eval_lyric)
lyric_df['hillarylength'] = lyric_df['Hillary100words'].apply(lambda x: len(x))

print (len(lyric_df[lyric_df['hillarylength'] == 100]))
print (type(lyric_df['Hillary100words'].iloc[0]))

141747
141259
<class 'list'>


In [128]:
lyricdf = lyric_df[lyric_df['hillarylength'] == 100]

In [129]:
lyricdf

,Unnamed: 0,genre,song,clean_lyrics,clean_song,language,lyrics,Hillarylyrics,Satorulyrics,Hillary100words,...,genre_Electronic,genre_Folk,genre_Hip-Hop,genre_Indie,genre_Jazz,genre_Metal,genre_Pop,genre_R&B,genre_Rock,hillarylength
0,0,Country,silver wings,silver wings shine in the sunlight roaring eng...,silver wings,English,Silver wings Shine in the sunlight. Roaring en...,silver wings shine in the sunlight roaring eng...,silver wings shine in the sunlight roaring eng...,"[flight, theyre, taking, you, away, leavin, me...",...,0,0,0,0,0,0,0,0,0,100
1,1,Country,she even woke me up to say goodbye,mornings come and lord my mind is aching that ...,she even woke me up to say goodbye,English,Morning's come and Lord my mind is aching That...,mornings come and lord my mind is aching that ...,mornings come and lord my mind is aching that ...,"[heart, is, silently, breaking, as, my, tears,...",...,0,0,0,0,0,0,0,0,0,100
2,2,Country,amazing grace,amazing grace how sweet the sound that saved a...,amazing grace,English,Amazing grace! How sweet the sound That save...,amazing grace how sweet the sound that saved a...,amazing grace how sweet the sound that saved a...,"[like, me, i, once, was, lost, but, now, am, f...",...,0,0,0,0,0,0,0,0,0,100
3,3,Country,every once in a while,every once in a while i think about you and yo...,every once in a while,English,Every once in a while I think about you And yo...,every once in a while i think about you and yo...,every once in a while i think about you and yo...,"[back, of, my, mind, when, i, hear, an, old, s...",...,0,0,0,0,0,0,0,0,0,100
4,4,Country,5-1-5-0,ever since i met you girl you been on my brain...,NaN,English,"Ever since I met you girl, you been on my brai...",ever since i met you girl you been on my brain...,ever since i met you girl you been on my brain...,"[bout, you, baby, just, this, side, of, loco, ...",...,0,0,0,0,0,0,0,0,0,100
5,5,Country,hard truth,you dont know who i e am but i know all about ...,hard truth,English,You Don't know who I [E] am But I know all abo...,you dont know who i e am but i know all about ...,you dont know who i e am but i know all about ...,"[could, have, had, it, a, all, a, family, and,...",...,0,0,0,0,0,0,0,0,0,100
6,6,Country,after the fire is gone,love is where you find it when you find no lov...,after the fire is gone,English,Love is where you find it When you find no lov...,love is where you find it when you find no lov...,love is where you find it when you find no lov...,"[place, again, we, know, its, wrong, for, us, ...",...,0,0,0,0,0,0,0,0,0,100
7,7,Country,a house with no curtains,we still were our rings we still say i love yo...,a house with no curtains,English,We still were our rings We still say I love yo...,we still were our rings we still say i love yo...,we still were our rings we still say i love yo...,"[but, everyone, knows, its, just, a, sad, show...",...,0,0,0,0,0,0,0,0,0,100
8,8,Country,wanderin',wanderin wanderin mhm my father is an engineer...,wanderin,English,(Wanderin' wanderin' mhm) My father is an eng...,wanderin wanderin mhm my father is an engineer...,wanderin wanderin mhm my father is an engineer...,"[mhm, my, father, is, an, engineer, my, brothe...",...,0,0,0,0,0,0,0,0,0,100
9,9,Country,katie bar the door,i could tell the way she held me out on that s...,katie bar the door,English,I could tell the way she held me Out on that s...,i could tell the way she held me out on that s...,i could tell the way she held me out on that s...,"[door, is, that, your, husband, coming, home, ...",...,0,0,0,0,0,0,0,0,0,100


In [130]:
lyricdf = lyricdf.drop(['hillarylength'],axis = 1)

In [131]:
lyricdf.columns

Index(['Unnamed: 0', 'genre', 'song', 'clean_lyrics', 'clean_song', 'language',
       'lyrics', 'Hillarylyrics', 'Satorulyrics', 'Hillary100words',
       'Satoru100words', 'artist', 'genre_Country', 'genre_Electronic',
       'genre_Folk', 'genre_Hip-Hop', 'genre_Indie', 'genre_Jazz',
       'genre_Metal', 'genre_Pop', 'genre_R&B', 'genre_Rock'],
      dtype='object')

## Create Test/ Training data set 

In [132]:
def split(df, test_size):
    data = df.values
    data_y = df.drop(df.iloc[:,0:12], axis=1).values
    #StratifiedShuffleSplit does not work with one hot encoded / multiple labels. Doing the split on basis of arg max labels.
    data_y = np.argmax(data_y, axis=1)
    data_y.shape
    stratified_split = StratifiedShuffleSplit(n_splits=2, test_size=test_size, random_state=42)
    for train_index, test_index in stratified_split.split(data, data_y):
        train, test = df.iloc[train_index], df.iloc[test_index]
    return train, test

In [133]:
train, test = split(lyricdf, 0.33)
#Split the train further into train and validation
train, validation = split(train, 0.2)

In [134]:
def generate_modelling_file(words):
    index = []
    for word in words:
        try:
            if lyric_tokenizer[word] > len(embedding_matrix) - 1:
                index.append( len(embedding_matrix) - 1)
            else:
                index.append(lyric_tokenizer[word])
        except:
            index.append( len(embedding_matrix) - 1)
            
    
    return embedding_matrix[index,:]

In [135]:
def create_label(row):
    return row[12:-1].values


In [136]:
test.iloc[0,12:-1].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=object)

In [137]:
train['X'] = train['Hillary100words'].apply(generate_modelling_file)
test['X'] = test['Hillary100words'].apply(generate_modelling_file)


In [138]:
test

,Unnamed: 0,genre,song,clean_lyrics,clean_song,language,lyrics,Hillarylyrics,Satorulyrics,Hillary100words,...,genre_Electronic,genre_Folk,genre_Hip-Hop,genre_Indie,genre_Jazz,genre_Metal,genre_Pop,genre_R&B,genre_Rock,X
11488,13326,Rock,gimme gimme gimme a man after midnight,half past twelve and im watching the late show...,gimme gimme gimme a man after midnight,English,Half past twelve And I'm watching the late sho...,half past twelve and im watching the late show...,half past twelve and im watching the late show...,"[a, fortune, to, win, its, so, different, from...",...,0,0,0,0,0,0,0,0,1,"[[-0.2708599865436554, 0.04400600120425224, -0..."
1085,1334,Jazz,brother [us version],brother theres so much that i could never say ...,brother us version,English,Brother There's so much that I could never sa...,brother theres so much that i could never say ...,brother theres so much that i could never say ...,"[place, how, do, i, find, words, that, do, not...",...,0,0,0,0,1,0,0,0,0,"[[-0.245059996843338, 0.04481000080704689, 0.3..."
5205,7190,Pop,de ware,ik kan je vragen om een vuurtje ik kan je vrag...,de ware,English,ik kan je vragen om een vuurtje ik kan je vrag...,ik kan je vragen om een vuurtje ik kan je vrag...,ik kan je vragen om een vuurtje ik kan je vrag...,"[moeite, loont, refr, maar, toen, ik, jou, hie...",...,0,0,0,0,0,0,1,0,0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3778,7941,Pop,we all pull the load,chorus we all pull the load we all pull the lo...,we all pull the load,English,[Chorus] We all pull the load We all pull the ...,chorus we all pull the load we all pull the lo...,chorus we all pull the load we all pull the lo...,"[in, the, road, that, hes, gotta, hold, oh, we...",...,0,0,0,0,0,0,1,0,0,"[[0.08570300042629242, -0.22201000154018402, 0..."
10465,20263,Pop,ae fond kiss,ae fond kiss and then we sever ae fareweel ala...,ae fond kiss,English,"Ae fond kiss and then we sever Ae fareweel, a...",ae fond kiss and then we sever ae fareweel ala...,ae fond kiss and then we sever ae fareweel ala...,"[tears, ill, pledge, thee, warring, sighs, and...",...,0,0,0,0,0,0,1,0,0,"[[0.409170001745224, 0.2860499918460846, 0.038..."
22939,26027,Indie,that's why,this guys complaining from the pages of his li...,thats why,English,This guys Complaining from the pages of his l...,this guys complaining from the pages of his li...,this guys complaining from the pages of his li...,"[complaining, from, the, pages, of, his, life,...",...,0,0,0,1,0,0,0,0,0,"[[0.2451999932527542, -0.1017799973487854, 0.4..."
17639,28040,Pop,i wanna touch you,ohhh yeayeah a redone colby o cherry cherry bo...,i wanna touch you,English,Ohhh!. Yea-yeah!. A RedOne!. Colby O!. Cherry ...,ohhh yeayeah a redone colby o cherry cherry bo...,ohhh yeayeah a redone colby o cherry cherry bo...,"[touch, you, but, im, trying, keep, control, o...",...,0,0,0,0,0,0,1,0,0,"[[-0.31036001443862915, -0.1800599992275238, 0..."
8161,14682,Rock,cosmic funeral,winged disciples of the nocturne dream descend...,cosmic funeral,English,Winged disciples of the nocturne dream Descend...,winged disciples of the nocturne dream descend...,winged disciples of the nocturne dream descend...,"[winged, disciples, of, the, nocturne, dream, ...",...,0,0,0,0,0,0,0,0,1,"[[-0.43665000796318054, 0.6520299911499023, 0...."
15652,19216,Rock,before the tears,e stevensp overstreet i dont want to see you c...,before the tears,English,(E. Stevens/P. Overstreet) I don't want to see...,e stevensp overstreet i dont want to see you c...,e stevensp overstreet i dont want to see you c...,"[start, to, fall, ooh, im, leaving, before, th...",...,0,0,0,0,0,0,0,0,1,"[[0.23336000740528107, 0.1604200005531311, 0.0..."
1425,1743,Hip-Hop,want u to be,oh yeah you know what baby ive been meaning to...,want u to be,English,(Oh Yeah) (You know what baby? Ive been meanin...,oh yeah you know what baby ive been meaning to...,oh yeah you know what baby ive been meaning to...,"[me, fellas, why, she, gotta, be, the, coldest...",...,0

In [139]:
train['Y_out'] = train.apply(lambda row: create_label(row), axis = 1)
test['Y_out'] = test.apply(lambda row: create_label(row), axis = 1)


In [140]:
np.shape(train['X'].iloc[1000])

(100, 100)

In [141]:
train['X'].iloc[1000]

array([[-0.071953  ,  0.23127   ,  0.023731  , ..., -0.71894997,
         0.86894   ,  0.19539   ],
       [-0.046539  ,  0.61966002,  0.56647003, ..., -0.37616   ,
        -0.032502  ,  0.80620003],
       [ 0.70212001,  0.70055002,  0.1104    , ...,  1.07270002,
         0.31071001, -0.056819  ],
       ...,
       [ 0.26572001,  0.034857  ,  0.49055001, ...,  0.39124   ,
         0.98232001,  0.069209  ],
       [ 0.27474001,  0.20158   ,  1.01629996, ...,  0.24530999,
        -0.27026999,  0.49112999],
       [-0.057078  ,  0.39873999,  0.68861002, ..., -0.19357   ,
        -0.030606  ,  0.2397    ]])

In [142]:
#np.shape(train['X'])
#np.shape(train['Y_out'])

print (np.shape(np.array(train['X'].tolist())))
print (np.shape(np.array(train['Y_out'].tolist())))

(75714, 100, 100)
(75714, 10)


In [143]:
train['Y_out'].iloc[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=object)

In [ ]:
model.fit(np.array(train['X'].tolist()),np.array(train['Y_out'].tolist()), epochs=10)
model.save_weights("newmodel.h5")
print("Saved model to disk")

model.load_weights("newmodel.h5")
print("Loaded model from disk")
 
x_test = np.array(test['X'].tolist())
y_test = np.array(test['Y_out'].tolist())

# evaluate loaded model on test data
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
score = model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))


Epoch 1/10
81642/81642 [==============================] - 563s 7ms/step - loss: 1.5404
Epoch 2/10
81642/81642 [==============================] - 525s 6ms/step - loss: 1.4045
Epoch 3/10
81642/81642 [==============================] - 527s 6ms/step - loss: 1.3607
Epoch 4/10
81642/81642 [==============================] - 526s 6ms/step - loss: 1.3249
Epoch 5/10
81642/81642 [==============================] - 527s 6ms/step - loss: 1.2960
Epoch 6/10
81642/81642 [==============================] - 527s 6ms/step - loss: 1.2675
Epoch 7/10
81642/81642 [==============================] - 526s 6ms/step - loss: 1.2414
Epoch 8/10
81642/81642 [==============================] - 541s 7ms/step - loss: 1.2118
Epoch 9/10
81642/81642 [==============================] - 544s 7ms/step - loss: 1.1867
Epoch 10/10
81642/81642 [==============================] - 528s 6ms/step - loss: 1.1619
Saved model to disk
Loaded model from disk
acc: 51.95%


Saved model to disk


Loaded model from disk
acc: 46.74%


In [144]:
y_pred = model.predict(np.array(train['X'].tolist()))

In [33]:
y_pred[0,:]

array([7.7515896e-03, 1.3183144e-02, 9.9187647e-04, 9.3381143e-01,
       3.4307441e-04, 1.0202437e-03, 3.4515900e-03, 2.5834098e-02,
       2.7899879e-03, 1.0822984e-02], dtype=float32)

In [34]:
y_train = np.array(train['Y_out'].tolist())
y_train[0,:]

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=object)

In [35]:
trues=0
for ind in range(0, len(y_pred)):
    if np.argmax(y_pred[ind,:]) == np.argmax(y_train[ind,:]):
        trues += 1
print ('accuracy', trues/len(y_pred))        
        
    

accuracy 0.5566547734890258


In [42]:
y_test_pred = model.predict(np.array(test['X'].tolist()))

In [43]:
trues=0
for ind in range(0, len(y_test_pred)):
    if np.argmax(y_test_pred[ind,:]) == np.argmax(y_test[ind,:]):
        trues += 1
print ('accuracy', trues/len(y_test_pred))        


accuracy 0.3971961249158545


In [25]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [26]:
generate_text("Dear Illinois department of insurance we love you and we hope that", 100, model, max_sequence_len)

"Dear Illinois Department Of Insurance We Love You And We Hope That I Could Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And I Can'T Be The And"

In [27]:
def train_random_batch(model, inp_sequences, epochs=1):
    for epoch in range(epochs):
        print("Epoch: {}".format(epoch+1))
        batch = np.random.choice(inp_sequences, size=10000)
        predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences[:100000])
        
        model.fit(predictors, label, epochs=1)

In [28]:
train_random_batch(model, inp_sequences)

Epoch: 1
Epoch 1/1
100000/100000 [==============================] - 315s 3ms/step - loss: 4.7141


In [29]:
generate_text("Dear Illinois department of insurance we love you and we hope that", 100, model, max_sequence_len)

"Dear Illinois Department Of Insurance We Love You And We Hope That I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know That I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know That I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know I Don'T Know That I Don'T Know I Don'T Know I"

#### Save model

In [8]:
def save_model(model, name):
    json_path = "keras_model/{}.json".format(name)
    weight_path = "keras_model/{}.h5".format(name)
    with open(json_path, "w") as f:
        f.write(model.to_json())
    model.save_weights(weight_path)
    return None

In [ ]:
#save_model(model, "0804_lunch")
#pickle.dump(tokenizer, open("keras_model/0804_tokenizer.pickle", "wb"))